<h1>The5Outliers - <span style="background: linear-gradient(to right, #28649c, #1dc2b1); -webkit-background-clip: text; color: transparent;">Wikispeedia Voyages</span>: why so many players pass through World regions to reach their target</h1>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import seaborn as sn
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import numpy as np

import config
from src.utils.HTMLParser import HTMLParser
from src.data.data_loader import *
from src.utils.helpers import *
from src.models.networks import *
from src.models.similarity_matrices import *
from src.models.markov_chains import *

parser = HTMLParser()
parser.load_pickle()

In [3]:
# Delete for final version?
# Compute and save all embending, similarity matrices and parsed articles
#compute_save_all()

# 0. Data preprocessing and loading

<div class="alert alert-block alert-info">

We use the `read_all()` function to load, clean, and enhance the data with useful features for analysis. It combines multiple data sources, performs preprocessing, and computes additional features for articles and user paths.

</div>

## Preprocessed Dataframes:
<div class="alert alert-block alert-info">

- `df_article_names`: Valid article names, that is without special characters and discardig any invalid articles (e.g. non-Wikispeedia articles or with missing categories).
- `df_html_stats`: Gathers wikispedia page statistics like number of links, link density, and structural information from the HTML version of each article.
- `df_categories`: Article categories and sub categories.
- `df_links`: Links inward and outward between articles.
- `df_shortest_path_length`: Matrix of shortest paths lengths possible between two articles.
- `df_unfinished` and `df_finished`: Cleaned paths with computed features.
- `df_sm` and `df_scat` (not returned): Matrices for semantic and category similarity.
- `df_article`: Dataframe with article-level features.

<div>


In [ ]:
# Load and preprocess the data
df_article_names, df_html_stats, df_categories, df_links, df_shortest_path, df_unfinished, df_finished, df_article = read_all()

## Key Modifications

### 1. Categories:
<div class="alert alert-block alert-info">
Each category is organized into multiple sub-levels, with a hierarchical depth of up to 3 levels. We observe that some articles are associated with multiple categories, highlighting overlaps and shared connections within the dataset. For our analysis we need to have only one category corresponding to each article, thus we choose to keep the most specific one.

Attributing the **first level category** based on the one **with fewer total articles** helps prioritize specialization over generality. 

Categories with fewer total articles are typically more specific, while those with higher counts cover broader topics. By focusing on the category with fewer articles, we ensure the article’s primary focus is on a unique or specialized perspective, providing a clearer thematic assignment. This method promotes a balanced classification system, ensuring articles are categorized accurately without being overshadowed by more general categories.

- For example, a category like **"Science"** may encompass a wide range of topics, while **"IT"** might be more specialized.
</div>

In [ ]:
df_categories_original = read_categories()
# Count the number of articles with multiples categorie
df_categories_original.groupby("article")["article"].size().value_counts()

In [ ]:
labels, parents, values, ids = create_treemap_data(df_categories_original, show_articles=True)

create_colored_treemap(labels, parents, values, ids,color_palette=config.PALETTE_CATEGORY_DICT_COLORS, title="Category Distribution in Articles (Only the most specific category is shown for each article)")
print(f'We have {df_categories_original["level_1"].unique().size} distinct level 1 categories.')

### 2. Article Features:

<div class="alert alert-block alert-info">

- **In-Degree**: of an article corresponds to the number of links on other pages targetting an article 
    - Some articles have no articles leading to it.
- **Out-Degree**: of an article corresponds to the number of links towards other pages on this article
    - Some articles lead to no other articles.
- **HTML Stats**: Structural details like the number of links per article.
    - Links do not consider duplicates inside the page (each link is considered to appear once) but this is relaxed with the HTML parsing.
- **Categories**: Assigns the most specific category to each article.
</div>

### 3. Path Features:
<div class="alert alert-block alert-info">
Metrics computed for user navigation paths:

- **Path Length**: Number of articles in a user's path.
- **Back-Clicks**: How many times users revisit previous articles.
- **Cosine Similarity**: Semantic similarity between source and target articles.
- **Shortest Path**: Length of the shortest path possible between the source and target articles.
- **Category Similarity**: Category similarity between the categories of the source and target articles.
</div>

## 1 Find Voyages
<div class="alert alert-block alert-info">

The goal of our research is to explore the role and significance of World Regions in user behavior while playing Wikispeedia. To achieve this, we first extract articles corresponding to *World Regions*.  
Fortunately, articles are categorized, so we gather those under *'Countries'* and relevant subcategories of *'Geography'* such as *'Geography of Asia'*. These articles are grouped into a new category called *'World Regions'*, while remaining *'Geography'* articles are keeping the same category.
</div>

In [7]:
world_regions_categories = ['Geography of Great Britain', 'Geography of Asia', 'Geography of Oceania Australasia', 'North American Geography', 'European Geography', 'African Geography', 'Central and South American Geography', 'Antarctica', 'Geography of the Middle East', 'African Countries', 'Countries']
df_categories = assign_world_region_categories(df_categories, world_regions_categories)

<div class="alert alert-block alert-info">

Below we can observe the new distributions of articles categories after the most specific category and the assignement of the new categorgories world regions.
</div>

In [ ]:
labels_filtered, parents_filtered, values_filtered, ids_filtered = create_treemap_data(df_categories, show_articles=True)
cat_fig = create_colored_treemap(labels_filtered, parents_filtered, values_filtered, ids_filtered,color_palette=config.PALETTE_CATEGORY_DICT_COLORS, title="Category Distribution in Articles (Only the most specific category is shown for each article)")

In [9]:
# Save the Plotly figure as a full HTML file
#cat_fig.write_html("html/categories.html", include_plotlyjs="cdn")

In [ ]:
plot_articles_pie_chart(df_categories, config.PALETTE_CATEGORY_DICT_COLORS, config.CATEGORY_ABBREVIATIONS)

### Sort games into voyage or not-voyage & assign the new category *'World Regions'*

<div class="alert alert-block alert-info">

A <span style="background: linear-gradient(to right, #3458d6, #34d634); -webkit-background-clip: text; color: transparent;">Wikispeedia Voyage</span> is defined as a path game where neither the source nor the target is in **World Regions**, but the path includes at least one article from this category.
</div>

In [10]:
# Sort paths into voyage and non-voyage
df_finished = game_voyage_sorting(df_finished, df_categories)
df_unfinished = game_voyage_sorting(df_unfinished, df_categories)

# Assign the categories "World Regions" to the concerned articles
category_map = dict(zip(df_categories["article"], df_categories["level_1"]))
df_article["category"] = df_article["article"].map(category_map)

In [ ]:
# Count the voyages 
voyage_count = (df_finished['Wikispeedia_Voyage'] == 1).sum() + (df_unfinished['Wikispeedia_Voyage'] == 1).sum()
non_voyage_count = (df_finished['Wikispeedia_Voyage'] == 0).sum() + (df_unfinished['Wikispeedia_Voyage'] == 0).sum()

print('Out of the ', len(df_finished)+len(df_unfinished), ' finished and unfinished games : ')
print('  - ', voyage_count, ' are voyages')
print('  - ', non_voyage_count, ' are not')

<div class="alert alert-block alert-success">

We immediately see that <span style="background: linear-gradient(to right, #3458d6, #34d634); -webkit-background-clip: text; color: transparent;">Wikispeedia Voyage</span> are very commun since they are approximately one third of the paths.

In [ ]:
#concatenate all games in a dataframe and plot the sankey diagram 
df_merged = pd.concat([df_finished, df_unfinished], ignore_index=True, sort=False)
sankey_fig = plot_sankey_voyage(df_merged, background_color="white")

In [13]:
# Save the Plotly figure as a full HTML file
#sankey_fig.write_html("html/sankey.html", include_plotlyjs="cdn")

# 2. Article features

## 2.1.a. Distributions of words, links, and categories in articles

In [ ]:
#TODO: big big plot voyage/non-voyage

metrics = [
    ("total_words", "Number of Words in Article"),
    ("abstract_words", "Number of Words in Article Abstract"),
    ("link_density", "Links Density in Article"),
    ("abstract_link_density", "Links Density in Article Abstract"),
    ("num_sections", "Number of Sections in Article"),
    ("num_subsections", "Number of Sub- Sections in Article")
]

fig, ax = plt.subplots(2, 3, figsize=(15, 5))

# Loop through metrics to create histograms
for i, (metric, title) in enumerate(metrics):
    row, col = divmod(i, 3)
    sn.histplot(df_article, x=metric, bins=30, kde=True, ax=ax[row, col])
    ax[row, col].set_title(title)
    if col == 1 or col == 2:
        ax[row, col].set_ylabel('')

plt.tight_layout()
plt.suptitle("Distribution of Article Metrics", y=1.05, fontsize=16)
plt.show()

<div class="alert alert-block alert-success">

Results : The data is right-skewed for most of metrics, indicating that while most articles adhere to certain standards of complexity and length, a small subset stands out as particularly detailed or interconnected.

## 2.1.b Articles Complexity

<div class="alert alert-block alert-info">

Which categories feature the most complex articles? To answer this question, let’s identify the most complex articles and the categories they belong to. We define complexity based on factors such as the number of words, links, and sections within each article.

In [ ]:
#add count of start and target in df_articles
count_start_and_target_per_articles(df_finished, df_unfinished, df_article)

#plot 
plot_proportion_category_start_stop_pies(
    df_article=df_article,
    palette=config.PALETTE_CATEGORY_DICT_COLORS,
    abbreviations=config.CATEGORY_ABBREVIATIONS,
    threshold=2.3
)

<div class="alert alert-block alert-success">

"Science" and "World Regions" categories are the most prevalent categories in both start and end articles, indicating their significant representation.

In [ ]:
metrics = [
    "total_words", "link_density", "num_sections",
    "abstract_words", "abstract_link_density", "num_subsections"
]

plot_metrics_by_category(df_article, metrics, config.PALETTE_CATEGORY_DICT_COLORS, config.CATEGORY_ABBREVIATIONS)

#TODO : add abreviations on x axis anf in legend 

In [ ]:
plot_proportions_of_in_and_out_degree_in_categories(
    df=df_article,
    palette=config.PALETTE_CATEGORY_DICT_COLORS,
    abbreviations=config.CATEGORY_ABBREVIATIONS
)

<div class="alert alert-block alert-success">

Results : The category "World Regions" stands out as a particularly complex topic, characterized by articles with the highest link density and the greatest number of category levels. World Regions articles are highly represented in the network, being the category with the highest number of incoming and outgoing links.

## 2.1.c Articles Popularity and Connectivity

<div class="alert alert-block alert-info">

Is there a correlation between the number of links in an article and the frequency with which people interact with it in the game? Does this depend more on the in-degree or out-degree of the links? And is there a significant difference between the in-degree and out-degree of links? We will compare and investigate these factors to understand their impact on article interaction.

In [ ]:
#TODO. comment + lists of countries in other categories
plot_article_popularity_link_density(df_article, df_finished, config.PALETTE_CATEGORY_DICT_COLORS, config.CATEGORY_ABBREVIATIONS, df_categories)

<div class="alert alert-block alert-success">

Results : We observe a significant overlap between the most visited articles and those with the highest degree. Additionally, the "Countries" category is prominently represented.

# 3. Analyse user behaviour (paths)

<div class="alert alert-block alert-info">

Let's look for patterns in user behaviour and try to understand how we could measure whether a game was difficult or not. Many different metrics can be considered, for instance:
- Game duration
- Game path length
- Difficulty rating given for finished paths
- Number of back-clicks needed
- Whether a game was finished or not
- For unfinished games, how the game was abandoned

A combination of these parameters can help finding in which games users struggled. This will then allow to assess whether players struggle less in <span style="background: linear-gradient(to right, #3458d6, #34d634); -webkit-background-clip: text; color: transparent;">Wikispeedia Voyages</span> than in other paths.

Let's first have a look of how the difficulty measures are distributed amongst each other for finished paths.

## 3.a Path duration

In [ ]:
#TODO: keep code but not plot, not link to story

# Find medians and kurtosis for both finished and unfinished
median_finished = df_finished['durationInSec'].median()
kurtosis_finished = kurtosis(df_finished['durationInSec'])
median_unfinished = df_unfinished.groupby('type')['durationInSec'].median()
kurtosis_unfinished = df_unfinished.groupby('type')['durationInSec'].apply(kurtosis)

print(f"The median duration of finished paths is {median_finished:.0f} seconds")
print(f"The median duration of all unfinished paths is {df_unfinished['durationInSec'].median():.0f} seconds, among which :")
print(f"  - The median duration of restart paths is {median_unfinished['restart']:.0f} seconds")
print(f"  - The median duration of timeout paths is {median_unfinished['timeout']:.0f} seconds")

print(f"Kurtosis of finished game durations: {kurtosis_finished:.2f}")
print(f"Kurtosis of restart game durations: {kurtosis_unfinished['restart']:.2f}")
print(f"Kurtosis of timeout game durations: {kurtosis_unfinished['timeout']:.2f}")

# Plotting histograms
plt.figure(figsize=(12, 4))

# Unfinished games
ax1 = plt.subplot(121)
sn.histplot(df_unfinished, x='durationInSec', hue='type', bins=100)
plt.yscale('log')
plt.xlabel('Duration in seconds')
plt.title('Histogram of game duration for unfinished games')

# Finished games
plt.subplot(122, sharey=ax1)
sn.histplot(df_finished, x='durationInSec', bins=100, alpha=0.5, color='y')
plt.yscale('log')
plt.xlabel('Duration in seconds')
plt.title('Histogram of game duration for finished games')
plt.show()

<div class="alert alert-block alert-success">

Time-out paths are games where the player stopped playing for more that 30 minutes. If we set those aside, and look at the duration of paths where the players decided to restart a new game, the mediam duration of games is 114 seconds, approximately the same as finished path duration of 107 seconds.

The kurtosis in finished games is much higher than in unfinished restart games (3973 vs 45), meaning that finished games are more heavy tailed. We can in fact see very little players restart after 3000 seconds, whereas many finished games go beyond 6000. 

## 3.b Path duration and path length vs. user rating

Let's first look at path duration distribution over different user rating. We found similar results for finished and unfinished paths, so let's have a look at **finished ones** for example.

In [ ]:
# change metric to 'path_length' for path length 
metric = 'durationInSec'

df_finished_strNaN = df_finished.copy()
df_finished_strNaN['rating'] = df_finished_strNaN['rating'].fillna('NaN')

plt.figure(figsize=(10, 6))
blues_palette = sn.color_palette("Blues", n_colors=6)

means, stds = [], []
ax1 = plt.subplot(231)
sn.histplot(df_finished[df_finished_strNaN['rating']=='NaN'], x=metric, bins=50, log_scale=True, color=blues_palette[0]) 
mean = df_finished[df_finished_strNaN['rating']=='NaN'][metric].mean()
plt.axvline(mean, color='red', label=f'Mean: {mean:.2f}', linestyle='--')
plt.yscale('log')
plt.xlabel('Duration in seconds')
plt.title('NaN')
plt.legend()

for i in range(1, 6):
    plt.subplot(231+i, sharex = ax1, sharey=ax1)
    sn.histplot(df_finished[df_finished_strNaN['rating']==i], x=metric, bins=50, log_scale=True, color=blues_palette[i])
    mean = df_finished[df_finished_strNaN['rating']==i][metric].mean()
    plt.axvline(mean, color="red", label=f'Mean: {mean:.2f}', linestyle='--')
    plt.yscale('log')
    plt.title(i)
    plt.xlabel('Duration in seconds')
    plt.legend()

plt.tight_layout()
plt.suptitle('Path duration by rating of the finished paths', y=1.04)
plt.show()

<div class="alert alert-block alert-success">

The distributions of the path duration by rating are nice log-normal distributions! This means we can use the mean (location) and standard deviation (scale) to characterise them. Indeed, except for NaN, there is a steady increase of the path duration mean when rating goes up.

<div class="alert alert-block alert-info">

Let's now look at whether path duration is correlated with path length:

In [ ]:
df_path_stats_duration = pd.DataFrame()
df_path_stats_duration['mean'] = df_finished.groupby('rating', dropna=False)['durationInSec'].mean()
df_path_stats_duration['std'] = df_finished.groupby('rating', dropna=False)['durationInSec'].std()
df_path_stats_duration['sem'] = df_finished.groupby('rating', dropna=False)['durationInSec'].sem()

df_path_stats_length = pd.DataFrame()
df_path_stats_length['mean'] = df_finished.groupby('rating', dropna=False)['path_length'].mean()
df_path_stats_length['std'] = df_finished.groupby('rating', dropna=False)['path_length'].std()
df_path_stats_length['sem'] = df_finished.groupby('rating', dropna=False)['path_length'].sem()

df_path_stats = pd.concat([df_path_stats_duration, df_path_stats_length], axis=1, keys=['duration', 'length'])

df_path_stats

In [ ]:
order = ['NaN', '1.0', '2.0', '3.0', '4.0', '5.0']

plt.figure(figsize=(10, 4))
plt.subplot(121)
sn.barplot(df_finished_strNaN, x='rating', y='durationInSec', order=order, errorbar=('ci', 95), color='#0b3880')
plt.xlabel('Rating')
plt.ylabel('Duration in seconds')
plt.subplot(122)
sn.barplot(df_finished_strNaN, x='rating', y='path_length', order=order, errorbar=('ci', 95), color='#0b3880')
plt.suptitle('Duration and Path Length by Rating of the Finished Paths')
plt.xlabel('Rating')
plt.ylabel('Path Length')
plt.show()

<div class="alert alert-block alert-success">

There is a clear correlation between how long players took or how many clicks they made with the experienced difficulty rating. This means we can easily combine the two into a difficulty measure as they agree with each other on what players considered difficult. 

Let's now loow at back-clicks: could it indicate whether players had a hard time?

## 3.c Back-clicks
<div class="alert alert-block alert-info">

The number of back-clicks made in a game may indicate players getting stuck.
Lets investigate how this metric is related with player ratings, and if they are more prominent in certain categories:

</div>

In [ ]:
# Define color palette with NaN explicitly included
blues_palette = sn.color_palette("Blues", n_colors=6)
rating_order_plot1 = ['1.0', '2.0', '3.0', '4.0', '5.0', 'NaN']  # For Plot 1
rating_order_plot2 = ['NaN', '1.0', '2.0', '3.0', '4.0', '5.0']  # For Plot 2 (stacked bars)
# Map ratings to consistent colors
rating_colors = {rating: blues_palette[i] for i, rating in enumerate(rating_order_plot2)}

# Function to extract categories where back-clicks occurred using Transition Category Path
def extract_back_click_categories(category_path):
    back_click_categories = []
    for i, category in enumerate(category_path):
        if category == '<' and i > 0:  # Detect back-click and ensure valid index
            back_click_categories.append(category_path[i - 1])  # Use preceding category
    return back_click_categories

# Apply the function to identify back-click categories
df_finished['back_click_categories'] = df_finished['Transition Category Path'].apply(extract_back_click_categories)

# Explode back-click categories for analysis
df_back_clicks = df_finished.explode('back_click_categories').dropna(subset=['back_click_categories'])
df_back_clicks['rating'] = df_back_clicks['rating'].fillna('NaN')
df_back_clicks['rating'] = df_back_clicks['rating'].astype(str)

# Count total back-click occurrences grouped by category and rating
category_rating_back_click_counts = (df_back_clicks.groupby(['back_click_categories', 'rating']).size().reset_index(name='back_click_count'))

# Count total occurrences of each category in Category Path
category_total_occurrences = df_finished['Category Path'].explode().value_counts().reset_index()
category_total_occurrences.columns = ['Category', 'total_occurrences']

# Merge total occurrences into back-click data
category_rating_back_click_counts = pd.merge(category_rating_back_click_counts,category_total_occurrences.rename(columns={'Category': 'back_click_categories'}),on='back_click_categories',how='left')

# Normalize back-click counts by total occurrences
category_rating_back_click_counts['normalized_back_clicks'] = (category_rating_back_click_counts['back_click_count']/ category_rating_back_click_counts['total_occurrences'])
# Reorder both columns (ratings) and rows (categories)
df_pivot = category_rating_back_click_counts.pivot(index='back_click_categories', columns='rating', values='normalized_back_clicks').fillna(0)
df_pivot = df_pivot[rating_order_plot2].loc[df_pivot[rating_order_plot2].sum(axis=1).sort_values(ascending=False).index]

fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
# ---- Plot 1: Back-Clicks per Rating ----
back_per_rating = (
    df_finished.groupby("rating", dropna=False)
    .agg(mean_back_clicks=("back_clicks", "mean"), mean_path_length=("path_length", "mean"))
    .reset_index()
)
back_per_rating['rating'] = back_per_rating['rating'].fillna('NaN')
back_per_rating['normalized_back_clicks'] = back_per_rating["mean_back_clicks"] / back_per_rating["mean_path_length"]

# Plot 1 with shared colors
# TODO : add eroor bars
sn.barplot(data=back_per_rating,x="rating",hue="rating",y="normalized_back_clicks",palette=[rating_colors[r] for r in rating_order_plot1],ax=ax[0])
ax[0].set_title("Distribution of Back-Clicks per Rating")
ax[0].set_xlabel("Rating")
ax[0].set_ylabel("Mean Back-Clicks number")

# ---- Plot 2: Stacked bar plot for back-clicks by category and rating ----
bottom = pd.Series([0] * len(df_pivot), index=df_pivot.index)
for rating in rating_order_plot2:
    ax[1].bar(df_pivot.index,df_pivot[rating],bottom=bottom,label=f'Rating {rating}',color=rating_colors[rating])
    bottom += df_pivot[rating]

ax[1].set_title("Normalized Back-Clicks by Category and Rating")
ax[1].set_xlabel("Category")
ax[1].set_ylabel("Normalized Back-Clicks")
ax[1].tick_params(axis='x', rotation=90)

# ---- Shared Legend ----
handles = [plt.Rectangle((0, 0), 1, 1, color=rating_colors[rating]) for rating in rating_order_plot1]
ax[0].legend(handles, [f"{r}" for r in rating_order_plot1], title="Rating", loc='upper left')
plt.tight_layout()
plt.show()

TO ADAPT A LITTLE WITH NEW PLOT
<div class="alert alert-block alert-success">

The back-clicks are correlated with the difficulty rating, which is expected as the more difficult a game is, the more likely players are to get stuck and go back.
The category with the most back-clicks is "Art", and the one with the least is "Countries". This is interesting as "Countries" is also one of the most visited category, which might indicate that players are more familiar with it and therefore less likely to get stuck.
Additionaly, The distribution of rating inside each category seams quite similar.

</div>

## 3.d Difficulty measure between voyage and non-voyage games

<div class="alert alert-block alert-info">

Here, we examine how difficulty metrics differ between  <span style="background: linear-gradient(to right, #3458d6, #34d634); -webkit-background-clip: text; color: transparent;">Wikispeedia Voyage</span> and non-Voyage paths, as well as between World Regions and non-World Regions categories.

</div>

In [ ]:
plot_difficulties_voyage(df_finished, df_unfinished, config.PALETTE_CATEGORY_DICT_COLORS)

In [ ]:
df_finished.groupby(["Wikispeedia_Voyage"])["back_clicks"].mean().reset_index()

<div class="alert alert-block alert-success">

 <span style="background: linear-gradient(to right, #3458d6, #34d634); -webkit-background-clip: text; color: transparent;">Wikispeedia Voyage</span> paths tend to be longer than non-voyage paths, with more back-clicks and a slightly higher rating. However, the percentage of completed games is higher for  <span style="background: linear-gradient(to right, #3458d6, #34d634); -webkit-background-clip: text; color: transparent;">Voyages</span> compared to Non-Voyages.

While World Regions articles are relatively few in number, they are highly prevalent within the network, often serving as key intermediate steps in paths. Although the <span style="background: linear-gradient(to right, #3458d6, #34d634); -webkit-background-clip: text; color: transparent;"> Voyage</span> paths seem effective, they tend to be slower and less efficient overall. Is the popularity of these articles solely due to this? Could it be that users intentionally choose these articles? Further investigation is needed to explore this.

</div>

## 3.2.a Transitions between categories (Networks)

<div class="alert alert-block alert-info">

Here, we create a network *(directed graph)* to represent user transitions between different categories of articles.

- **Nodes:** Each node represents a group of articles belonging to a specific category, with each article assigned to his main category. 
- **Edges:** Each edge represents a transition from one article category to another, indicating the flow of user activity.

In this visualization, we exclude transitions within the same category (self-loops), as these are already represented in the transition matrix shown earlier (3.3.b). The focus here is on how users navigate between different categories.

The node positions are determined using a **force-directed algorithm**:

- Nodes repel each other like charged particles, preventing overlaps.
- Edges act like springs, pulling connected nodes closer to reflect their relationships.

----------

</div>


<div class="alert alert-block alert-info">

In the cell below, 
- we recursively replace each back click **(<)** with the article that was visited prior to it. This ensures that we accurately reconstruct the original navigation path and correctly identify the corresponding category for each step 
- and we use the merged df of paths as in this section we will first only analyse how users make moves between categories of articles, independ of their succes on the research.

</div>

### User networks

In [ ]:
df_merged_no_backclick = df_merged['path'].apply(replace_back_clicks)
G_user = build_network_from_paths(df_merged_no_backclick, df_categories, include_self_loops=False)
plot_network(G_user, df_categories,palette=config.PALETTE_CATEGORY_DICT_COLORS, title="Network of transitions whithin Categoires", show_edge_labels=False, node_abbreviations=config.CATEGORY_ABBREVIATIONS)

### Structure wikipedia network

In [36]:
markov_transitions_cat = get_transition_probabilities(df_article_names, parser, backclicks=False, normalise=False)
markov_transitions_cat = pd.DataFrame(markov_transitions_cat)
markov_transitions_cat.index = df_article_names
markov_transitions_cat.columns = df_article_names

In [ ]:
G_structure = build_network_from_matrix(markov_transitions_cat,category_map,include_self_loops=False)
plot_network(G_structure, df_categories,palette=config.PALETTE_CATEGORY_DICT_COLORS, title="Network of transitions whithin Categoires", show_edge_labels=False, node_abbreviations=config.CATEGORY_ABBREVIATIONS)

## 3.3 Comparison with optimal paths

<div class="alert alert-block alert-info">

Let's define the optimal path as the shortest path that can be taken. This assumption is coherent since the goal of the game is to reach the target while keeping the number of clicks as low as possible.

We can analyze whether users’ choice of categories aligns with the categories in the optimal path to determine if passing through 'World Regions' is an effective strategy for reaching the target.

To achieve this, we first need to compute the optimal path by constructing a directed graph where edges represent connections between articles. Here we take all the shortest possible paths per each combinaton of source-target present in the users' games. 

</div>

In [29]:
#optimal_paths = compute_optimal_paths(df_links, df_shortest_path, df_finished, df_unfinished)

# Read precomputed optimal paths
optimal_paths = pd.read_pickle('data/paths-and-graph/optimal_paths.pkl')

<div class="alert alert-block alert-success">

Some combination of source and target have many possible shortest paths!

</div>

In [ ]:
# Count the number of shortest paths for each (source, target) combination
path_counts = optimal_paths.groupby(['source', 'target']).size().reset_index(name='path_count')
path_counts.sort_values(by='path_count')

In [31]:
# Map the paths artciles to their categories
optimal_paths['Category Path'] = optimal_paths['path'].apply(lambda path: map_path_to_categories(path, category_map))

# Get users paths mapped to categories and remove backclicks
users_finished, users_unfinished = users_paths(df_finished, df_unfinished, category_map)

# Filter optimal paths to match source-target pairs in users paths and remove path where target available in source
optimal_fin, users_finished, optimal_unf, users_unfinished = filter_pairs(optimal_paths, users_finished, users_unfinished)

<div class="alert alert-block alert-info">

To compare user paths with optimal paths, we calculate the percentage of times each category is visited at each step for both datasets and each source-target association.

1. We can average the percentages by groupping them by source-target category pairs (e.g., source in 'Art' and target in 'Music'), then average within these groups to balance category representation. We can then visualize the categories taken at each of the first 10 steps.

2. To examine the difference between user and optimal paths in general trends, we average these percentages across steps for each category. Next, we compute the difference between the optimal and user paths for each source-target association and category. We then average these differences by groupping them by source-target category pairs.

</div>

In [32]:
S_T_opt_fin_percentages, S_T_fin_percentages, S_T_opt_unf_percentages, S_T_unf_percentages = calculate_step_percentages(optimal_fin, users_finished, optimal_unf, users_unfinished)

dataframes = [
    (S_T_opt_fin_percentages, 'opt'),
    (S_T_fin_percentages, 'fin'),
    (S_T_opt_unf_percentages, 'opt'),
    (S_T_unf_percentages, 'unf')
]

category_fin_means, category_unf_means=process_and_calculate_differences(dataframes, category_map, column_names=['source', 'target', 'categories'])

In [ ]:
#TODO: warning
S_T_fin_percentages_norm_steps, S_T_opt_fin_percentages_norm_steps = process_percentages(S_T_opt_fin_percentages, S_T_fin_percentages, category_map, max_step=10)

<div class="alert alert-block alert-info">

This analysis applies to both finished and unfinished user paths. However, for easier visualization, we plot only finished paths at each step. The observed differences confirm that trends remain similar across both path types.

</div>

In [ ]:
plot_position_line(S_T_fin_percentages_norm_steps, S_T_opt_fin_percentages_norm_steps, category_fin_means, category_unf_means, config.PALETTE_CATEGORY_DICT_COLORS)

<div class="alert alert-block alert-success">

In early path positions, the category 'World Regions' appears most frequently and consistently decreases over subsequent steps, aligning with other categories. This trend holds true for both optimal and user paths.

Note that, optimal paths are often very short. After the 5th step, only a few paths remain, which predominantly include categories like 'Citizenship', causing their percentages to increase sharply at the end.

The plot clearly shows that users pass through 'World Regions' less often than optimal paths suggest. This finding was surprising, as we intuitively expected humans to use more  <span style="background: linear-gradient(to right, #3458d6, #34d634); -webkit-background-clip: text; color: transparent;">Wikispeedia Voyages</span> than necessary. However, the opposite is true. Users might benefit from using this category more, likely because, as shown earlier, it is highly interconnected with other categories.

</div>

## 3.4 User behaviour vs. random network transitions

<div class="alert alert-block alert-info">

There seems to be some inherent bias of the categories just by the structure of the Wikispeedia network. Indeed, it seems much more likely to end up in World Regions or Science just because the categories are much bigger and more links lead to it. Maybe there is a way of disentangling the user behaviour from the network structure? If we consider that users only click on links randomly (i.e. a random walk in the Wikispeedia Network), we can capture the effect of the network structure. Indeed, if taking a random walk always leads you to World Regions, then maybe the users do not actually choose to go there but the network structure just makes it very likely that they end there anyway.

To model this, Markov chains come in handy: we can model the transition probabilities from one article A to an article B just by counting the number of links to article B in article A, and dividing by the number of links there are on the page in total. While this does not eliminate all bias (for instance, links further up in the page may be clicked more), this Markov chain still gives valuable insight about the network structure.

To get an equivalent transition matrix for users, we can simply count the transitions users actually made at each step. To compare the two obtained probability distributions, we will use the Kullback–Leibler ($KL$) divergence, a type of statistical distance measuring how much a model probability distribution $Q$ (the random probabilities) differ from a probability distribution $P$ (our users' transitions). The KL-divergence is defined as

$$ D_{KL}(P || Q)_{ij} = P_{ij} \log \frac{P_{ij}}{Q_{ij}} $$


It is 0 when $P$ and $Q$ are exactly the same or when the user transition probability is 0. A higher value indicates a stronger divergence: if the divergence is high, users choose a certain transition more than the random transitions in the network would suggest. That is, the users actually choose this transition and it is not only due to the network structure.
</div>

In [ ]:
df_paths = pd.concat([df_finished[['path', 'path_length']], df_unfinished[['path', 'path_length']]])

markov_transitions = get_transition_probabilities(df_article_names, parser, backclicks=False)
user_transitions = get_nth_transition_matrix(df_finished, df_article_names, 1)

print(f'Non-zero values of random transitions: {np.count_nonzero(markov_transitions)}')
print(f'Non-zero values of user transitions: {np.count_nonzero(user_transitions.to_numpy())}')

<div class="alert alert-block alert-info">

First, let's see whether the obtained transition matrix makes sense with what we have observed previously. Let's have a look at which articles we seem to converge naturally to when taking a random walk in the Wikispeedia network.

The transition matrix's $P$ left eigenvector with eigenvalue $1$ (i.e. $x$ such that $xP=x$) is called the steady-state of the system. Let's call the normalised vector $x = (x_1, x_2, \dots, x_N)$, where the sum of all $x_i$ is $1$. After $\infty$ iterations, starting from whatever article, the probability distribution of the end article is given by $x$. 

**Example**: For 3 articles, let's say that we have $x=(0.2, 0.3, 0.5)$. Then, when starting from any of the articles and taking infinitely many steps, there is probability 0.2 to be in article 1, probability 0.3 to be in article 2 and probability 0.5 to be in article 3.

</div>

In [ ]:
# Markov steady-state with back-clicks
markov_transitions = get_transition_probabilities(df_article_names, parser, backclicks=True)
ssb = compute_steady_state(markov_transitions, df_article_names, backclicks=True)
ssb.sort_values(by='steady_state_proportion', ascending=False).head(5)

In [ ]:
# Markov steady-state with back-clicks
markov_transitions = get_transition_probabilities(df_article_names, parser, backclicks=False)
ss = compute_steady_state(markov_transitions, df_article_names)
ss.sort_values(by='steady_state_proportion', ascending=False).head(20)

<div class="alert alert-block alert-info">

We considered 2 scenarios: one with a "back-click article", which appears once on every page and therefore has a non-null probability of being chosen. The transition from this "back-click article" leads uniformly to all articles, as we make the assumption that every article is equally likely to be reached from a back-click. While this is certainly not true, it is an easy hypothesis to make in a system that is agnostic of player behaviours. The other scenario removes the possibility of using backclicks.

To find the probabilities to be in a certain article in the steady state, we can look at the normalised values of the eigenvector. In the first scenario, we see that with our assumptions, back-clicks are very favoured by the network (more than $2\%$ probability to back-click). This makes sense, as it is present on every article in Wikispeedia. 

In the second scenario, United States is the most likely article to be in ($1.35\%$ probability), followed mainly by other countries. It looks like countries are naturally favoured by the network structure. This is interesting as it shows that there is definitely a bias towards World Regions (particularly large countries) in Wikispeedia articles: they have more links and are more interconnected than other articles. This is a first hint that World Regions are intertwined with the human mind, as humans wrote the articles in questions and made them particularly rich and link-dense.

</div>

<div class="alert alert-block alert-success">

To answer our initial question (whether users choose World Region articles more than random) we need to compare the random transitions with the user transitions through the $KL$ divergence introduced above.
</div>

In [ ]:
KL = np.where((user_transitions > 0) & (markov_transitions > 0), user_transitions * np.log(user_transitions / markov_transitions), 0)
KL_df = pd.DataFrame(KL, columns=user_transitions.columns, index=user_transitions.index)

mean_KL = pd.DataFrame()
mean_KL['mean'] = KL_df.mean(axis=0)
mean_KL['CI'] = 1.96 * KL_df.sem(axis=0)
mean_KL.sort_values(by='mean', ascending=False).head(10)

In [ ]:
sn.heatmap(KL_df.iloc[4200:4300, 4200:4300], cmap='BuPu', cbar=True)
plt.ylabel('Pre-Transition Article')
plt.ylabel('Post-Transition Article')
plt.title(r'$KL$ Divergence for a Subset of Articles')
plt.show()

<div class="alert alert-block alert-info">

It seems that even when comparing the random transitions (that already favour World Region articles) with the user transitions, there is a consistent bias towards World Region articles. This shows that users use certain articles more than the already favourable random transitions would suggest. This could have multiple reasons: it could be because users are more comfortable with the articles' topic, but it could also be as part of the users' strategy, if users noticed that these articles had promising connectivity.

</div>

In [40]:
%%capture
mean_KL_step = get_step_divergences(df_article_names, parser, df_paths, backclicks=False)

In [ ]:
PALETTE_ARTICLE_DICT_COLORS = {}
for _, row in df_categories.iterrows():
    article = row['article']
    category = row['level_1']
    PALETTE_ARTICLE_DICT_COLORS[article] = config.PALETTE_CATEGORY_DICT.get(category, '#808080')

PALETTE_ARTICLE_DICT_COLORS['<'] = '#000000'

main_mean_div = mean_KL_step.iloc[:15]
fig_art = plot_article_step_divergence(main_mean_div, PALETTE_ARTICLE_DICT_COLORS)
fig_art.show()

In [42]:
%%capture
KL_cat_df = get_step_divergences(df_article_names, parser, df_paths, df_categories=df_categories, backclicks=False)

In [ ]:
backclick_palette = config.PALETTE_CATEGORY_DICT_COLORS
backclick_palette['<'] = '#000000'
fig_cat = plot_category_step_divergence(KL_cat_df, backclick_palette)
fig_cat.show()

<div class="alert alert-block alert-success">

We can take two main conclusions from these figures:
1. In the first few steps, World Region articles seem to be chosen by users much more than random. The Divergence Value is statistically significantly higher for World Regions than it is for all the other categories.
2. Certain articles in the World Region category have a much higher impact than others. Looking at the scales of mean divergence ($10^{-1}$ for United States vs $10^{-4}$ for aggregated World Regions), it seems that a few articles contribute much more than others to World Regions being so popular.

</div>

### Categories normalised by transition probabilities

In [ ]:
# shows that World Region articles are highly connected, but we can remove this plot if we are sure that we already show this elsewhere.

from collections import Counter
from src.utils.helpers import map_path_to_categories

markov_transitions = get_transition_probabilities(df_article_names, parser, backclicks=False, normalise=False)
article_to_category = dict(zip(df_categories['article'], df_categories['level_1']))
mapped_categories = map_path_to_categories(df_article_names, article_to_category)
transitions_per_category = pd.DataFrame({'category': mapped_categories, 'transition': markov_transitions.sum(axis=1)})
transitions_per_category=transitions_per_category.groupby('category', as_index=False).sum()
category_counts = Counter(mapped_categories)
category_counts_df = pd.DataFrame(category_counts.items(), columns=['category', 'count'])

result = pd.merge(category_counts_df , transitions_per_category, on='category')
result['ratio'] =  result['transition'] / result['count']
result = result.sort_values(by='ratio')

sn.barplot(data=result, x='category', y='ratio', hue='category', legend=None, palette='viridis')

# Customize the plot
plt.xlabel('Category')
plt.ylabel('Transitions / # Articles in Category')
plt.title('Ratio of Count to Transitions by Category')
plt.xticks(rotation=90) 
plt.show()

### Page structure

<div class="alert alert-block alert-info">

To relax the assumption made for Markov networks that every link in an article is equally likely to be clicked, we can have a look at how link position influences whether links are clicked or not. If we find for instance that only the first few links are generally clicked, the uniform link choice assumption may be false.

</div>

In [ ]:
# where are link to a certain category in a page ? TODO: Takes first link or the mean?
link_per_cat = find_category_position_articles(parser, df_categories, config.CATEGORIES_OTHERS)
df = pd.DataFrame(link_per_cat)
df.describe()

In [ ]:
# Where do people click on an article page ?
df_merged = location_click_on_page(df_merged, parser)

In [ ]:
# put that in a function

df_merged["category"] = "All"
df_merged["Legend :"] = "Clicked Link Position in Paths"
df_melted = pd.melt(df, var_name='category', value_name='position').dropna()
df_melted["Legend :"] = "Article Link Position in Articles"
df_comparison_path_category = pd.concat([df_merged[["category", "position", "Legend :"]], df_melted])

fig = px.box(df_comparison_path_category, x="category", y="position", color="Legend :", title="Position of the clicked link in articles compared to position of each category in articles")
fig.update_xaxes(tickangle=45)

fig.update_layout(
    autosize=False,
    width=1500,
    height=500,
    boxgroupgap=0.2, # update
    boxgap=0)
fig.show()

## 4.2. Evolution of similarity on paths

<div class="alert alert-block alert-info">

We can get an idea of how similar one article is to another by comparing the cosine similarity of the article name embeddings. While this is not a precise method (for individual paths, the similarity may or may not make intuitive sense), it seems that the aggregated similarity over all paths does give significant results. In the following we analyse the mean similarity between one article and the next in user paths for both Voyages and Other paths. For this, we use both BERT and BGEM3 embeddings, to validate our results with at least two different models. 

</div>

In [51]:
bert_similarity = read_similartiy_matrix(model='bert')
BGEM3_similarity = read_similartiy_matrix(model='BGEM3')

df_paths = pd.concat([df_finished[['path', 'Wikispeedia_Voyage']], df_unfinished[['path', 'Wikispeedia_Voyage']]])

bert_similarities = [
    get_path_similarities(df_paths[df_paths['Wikispeedia_Voyage']==True], df_sm=bert_similarity),
    get_path_similarities(df_paths[df_paths['Wikispeedia_Voyage']==False], df_sm=bert_similarity)
]

BGEM3_similarities = [
    get_path_similarities(df_paths[df_paths['Wikispeedia_Voyage']==True], df_sm=BGEM3_similarity),
    get_path_similarities(df_paths[df_paths['Wikispeedia_Voyage']==False], df_sm=BGEM3_similarity)
]

<div class="alert alert-block alert-info">

The similarities are computed for all paths and all steps along each path. We focus here on the first 10 steps of all paths and take the mean over each step in all paths. Because the paths have different lenghts, the mean of e.g. position L is taken over all paths that have at least L steps. Furthermore, we compute the interval as $1.96$ times the standard error, which we compute in the same fashion as the mean for each path position. We also normalise the scale to $[0, 1]$, as the similarity scores for the two models are in a different, somewhat arbitrary, range. To preserve linearity, we simply use min-max scaling for each type of embedding, setting the maximum mean similarity to 1 and the minimum to 0. 
</div>

In [52]:
bert_normalised_means, bert_normalised_sems = get_normalised_mean_similarity(bert_similarities, 10)
BGEM3_normalised_means, BGEM3_normalised_sems = get_normalised_mean_similarity(BGEM3_similarities, 10)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)
colours = ['#d9630a', '#6c2e02', '#1dc2b1', '#096057']
labels = ['Voyage Similarities (BERT)', 'Other Similarities (BERT)', 'Voyage Similarities (BGEM3)', 'Other Similarities (BGEM3)']
means = bert_normalised_means + BGEM3_normalised_means
sems = bert_normalised_sems + BGEM3_normalised_sems

for i in range(4):
    sn.lineplot(x=np.arange(1, len(means[i])+1), y=means[i], lw=3, color=colours[i], label=labels[i], ax=axs[i//2])

    axs[i//2].fill_between(
        np.arange(1, len(means[i])+1), 
        np.array(means[i][:10]) - np.array(sems[i][:10]), 
        np.array(means[i][:10]) + np.array(sems[i][:10]), 
        color=colours[i], alpha=0.3)

axs[0].set_title('BERT embedding')
axs[0].set_ylabel('Normalised Mean Similarity')
axs[0].set_xticks(np.arange(1, len(means[0])+1))
axs[0].set_xlabel('Step Along Path')

axs[1].set_xticks(np.arange(1, len(means[0])+1))
axs[1].set_xlabel('Step Along Path')
axs[1].set_title('BGEM3 embedding')

plt.suptitle('Mean Name Similarity Between Previous and Next Articles in User Paths')
plt.tight_layout()
plt.legend()
plt.show()

<div class="alert alert-block alert-info">

There is a clear trend for both types of embeddings: in the first few clicks, the articles chosen have a low similarity to the previous one. This can be interpreted as leaving the original category, because article names in different categories are likely to be less similar. After this, the similarity for the next clicks stabilises, with relatively small fluctuations.

There is also a statistically significant difference between Voyages and Other paths. Generally, both embeddings agree that the similarity along paths is generally lower for Voyages. This could indeed show that there is a stronger detour or zoom-out behaviour from users in Voyages, as the articles visited are on average less similar to one-another.

</div>